# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [23]:
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
j = r.json()
rio_sujo = j['items']
rio = pd.DataFrame(rio_sujo)
rio['status'].value_counts()
#Deferido = 91
#Arquivado = 53
#Regularizado = 25

Deferido                                                 91
Arquivado                                                53
Regularizado                                             25
Arquivado por decurso de prazo                           23
Deferido (PVL-IF)                                        16
Encaminhado à PGFN com manifestação técnica favorável    13
Arquivado a pedido                                        6
Em retificação pelo interessado                           3
Em análise                                                1
Em retificação pelo interessado (PVL-IF)                  1
Pendente de regularização                                 1
Name: status, dtype: int64

In [38]:
rio['data_status']
rio['ano'] = rio['data_status'].str.slice(start = 6, stop = None, step = 1)


In [42]:
rio['ano'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2022    12
2011    10
2009     9
2015     9
2017     5
2021     5
2016     4
2018     4
2006     4
2002     4
2004     1
2003     1
Name: ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [51]:
def consulta(url:str): #definição da função
    import pandas as pd #importando as bibliotecas
    import requests as req
    j = req.get(url) # pegando a url
    Dados = j.json() # recebendo os dados
    Dados = Dados['items'] # recebendo a lista
    data = pd.DataFrame(Dados) # transformando a lista no dataframe
    return data # retornando o dataframe

In [83]:
mg = consulta('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado')
mg['status'].value_counts()
#status = mg[mg['status'] == 'Arquivado por decurso de prazo']
#status.value_counts()


Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado pela STN                                        3
Arquivado                                                 3
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [82]:
ba = consulta('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio')
cond = ba['status'] == 'Deferido'
ba_deferido = ba[cond]
ba_deferido['interessado'].value_counts()
#Luís Eduardo Magalhães com 14

Luís Eduardo Magalhães    14
Vitória da Conquista      11
Lauro de Freitas          10
Camaçari                  10
Barreiras                  8
                          ..
Salinas da Margarida       1
Lençóis                    1
Nilo Peçanha               1
Jeremoabo                  1
Itapicuru                  1
Name: interessado, Length: 173, dtype: int64

In [91]:
ba = consulta('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio')
cond = ba['interessado'] == 'Salvador'
ba_estado = ba[cond]
ba_estado.to_csv('salvador.csv')

